<a href="https://colab.research.google.com/github/bryh311/FreeCodeCamp-ML/blob/main/bryce_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import string
import re
from keras.preprocessing import sequence

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
BUFFER_SIZE = 1000
BATCH_SIZE = 32
MAX_LEN = 250

# I took a lot of this from https://saturncloud.io/blog/how-to-convert-pandas-dataframes-to-tensorflow-batchdatasets-for-nlp/

def preprocess(text):
  text = text.translate(str.maketrans('', '', string.punctuation))
  text = text.lower()
  words = re.findall(r'\b\w+\b', text)
  return words

# read data into dataframes and change ham/spam
train_df = pd.read_csv(train_file_path, sep='\t')
train_df.columns = ['label', 'text']
train_df['label'] = train_df['label'].astype('category').cat.codes
train_df['text'] = train_df['text'].apply(preprocess)

test_df = pd.read_csv(test_file_path, sep='\t')
test_df.columns = ['label', 'text']
test_df['label'] = test_df['label'].astype('category').cat.codes
test_df['text'] = test_df['text'].apply(preprocess)

vocab = set()

for text in train_df['text']:
  vocab.update(text)

for text in test_df['text']:
  vocab.update(text)

word2idx = {}

for i, word in enumerate(vocab):
  word2idx[word] = i

VOCAB_SIZE = len(vocab) + 1

train_df['text'] = train_df['text'].apply(lambda x: [word2idx[word] for word in x])
test_df['text'] = test_df['text'].apply(lambda x: [word2idx[word] for word in x])

#train_label = train_df.pop('label')
#test_label = test_df.pop('label')

#train_dataset = tf.data.Dataset.from_tensor_slices((train_df['text'].values, train_df['label'].values))
#test_dataset = tf.data.Dataset.from_tensor_slices((test_df['text'].values, test_df['label'].values))

for i, arr in enumerate(train_df['text']):
  train_df['text'][i] += [0] * (MAX_LEN - len(train_df['text'][i]))
  train_df['text'][i] = np.array(train_df['text'][i])

for i, arr in enumerate(test_df['text']):
  test_df['text'][i] += [0] * (MAX_LEN - len(test_df['text'][i]))
  test_df['text'][i] = np.array(test_df['text'][i])

train_dataset = tf.data.Dataset.from_tensor_slices((np.stack(train_df['text'].values), train_df['label'].values))
test_dataset = tf.data.Dataset.from_tensor_slices((np.stack(test_df['text'].values), test_df['label'].values))

In [ ]:
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_df))

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 24),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(loss="binary_crossentropy",optimizer="adam", metrics=["accuracy"])

In [ ]:
#history = model.fit(np.stack(train_df['text'].values), train_df['label'], epochs=10)
history = model.fit(train_dataset, epochs=20, verbose=True)

In [ ]:
results = model.evaluate(np.stack(test_df['text'].values), test_df['label'])
print(results)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])


def to_num(word_list):
  ret = []
  for word in word_list:
    if word in word2idx.keys():
      ret.append(word2idx[word])
    else:
      ret.append(0)
  return ret

def predict_message(pred_text):
  input = preprocess(pred_text)
  nums = to_num(input)
  nums += [0] * (MAX_LEN - len(nums))
  nums = np.array([nums])
  #print(nums)
  res = model.predict(nums)

  text = "ham"

  if res[0] > 0.5:
    text = "spam"

  prediction = [res[0].tolist()[0], text]


  return (prediction)

pred_text = "you have won £1000 cash! call to claim your prize."

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
